In [1]:
from IPython.display import HTML

HTML('''<script>
code_show=false; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Show code"></form>''')

<a id='index'></a>
# Index
- [Base case](#base_case) - original Example 1 from Lakhdar et al. (2005)
- [Increased demand for product p1](#increased_p1_demand)
- [Doubled demand profile](#doubled_demand_profile)

In [2]:
import qgrid
qgrid.nbinstall(overwrite=True)

import pandas as pd
pd.options.display.max_rows = 999
pd.options.display.max_columns = 999

import plotly
import plotly.plotly as py
import plotly.figure_factory as ff
import plotly.graph_objs as go

In [ ]:
# Run the following command once to set up your Plotly account to enable plotting
# plotly.tools.set_credentials_file(username='DemoAccount', api_key='lr1c37zw81')

In [3]:
def make_gantt(schedule, title, start_date='2017-01-01'):        
    df = schedule.reset_index()
    
    for col in ['Start', 'Finish']:
        df[col] = df[col].apply(lambda x: (pd.Timedelta('%d days' % x) + pd.to_datetime(start_date)).date())
        
    df = df.to_dict('records')
    
    colors = ['rgb(244, 66, 66)', 'rgb(65, 244, 92)', 'rgb(73, 65, 244)']

    gantt = ff.create_gantt(df,
                            title=title, 
                            colors=colors,
                            showgrid_x=True,
                            index_col='Resource', 
                            group_tasks=True, 
                            show_colorbar=True)
    
    for gantt_row, schedule_row in zip(gantt['data'], df):
        text = "No. batches produced: %d<br>Total duration: %d days" \
            % (schedule_row['Batches'], (schedule_row['Finish'] - schedule_row['Start']).days)
            
        gantt_row.update({'text': text})
        
    return gantt

def make_demand_supply_chart(sales_profile, demand_data, title):
    x = ['t%d' % i for i in range(1, len(demand_data) + 1)]

    trace1 = go.Bar(
        x=x,
        y=demand_data,
        name='Demand',
        opacity=0.6
    )

    trace2 = go.Bar(
        x=x,
        y=sales_profile,
        name='Supply',
        opacity=0.6
    )

    layout = go.Layout(
        title=title,
        yaxis=dict(
            title='No. batches'),
        xaxis=dict(
            title='Time period')
    )

    return go.Figure(data=[trace1, trace2], layout=layout)

<a id='base_case'></a>
## Base case
A continuous-time, genetic algorithm based solution to the original **Example 1 problem** from *"[Medium term planning of biopharmaceutical manufacture using mathematical programming. Biotechnology progress](http://onlinelibrary.wiley.com/doi/10.1021/bp0501571/full)"* by Lakhdar et al. (2005).

[back to index](#index)

In [4]:
num_usp_suites = 2
num_dsp_suites = 2

print("No. USP Suites: %d, no. DSP Suites: %d" % (num_usp_suites, num_dsp_suites))

No. USP Suites: 2, no. DSP Suites: 2


In [5]:
product_data = pd.read_csv('data/product_data.csv', index_col='Product')
product_data

,USP rate [days],USP lead [days],USP lifetime [days],USP cap. per 60 days,DSP rate [days],DSP lead [days],DSP lifetime [days],DSP cap. per 60 days,USP storage cost,Prod. cost,Waste disp. cost,Sales price,DSP storage cost,Backlog penalty,Changeover cost
Product,,,,,,,,,,,,,,,
p1,20.0,10,60,10,10,10.0,180,40,5,2,5,20,1,20,1
p2,22.2,10,60,10,10,10.0,180,40,5,2,5,20,1,20,1
p3,12.5,10,60,10,10,12.5,180,40,5,2,5,20,1,20,1


In [6]:
days_per_period = [60, 60, 60, 60, 60, 60]
demand_data = pd.read_csv('data/demand_data.csv', index_col='Product')
qgrid.show_grid(demand_data)

Widget Javascript not detected.  It may not be installed or enabled properly.


In [7]:
from lakhdar2005_continuous import Example1Model

example1_model = Example1Model().fit(
    num_usp_suites,
    num_dsp_suites,
    
    demand_data.values,
    days_per_period,
    
    product_data['USP storage cost'],
    product_data['Sales price'],
    product_data['Prod. cost'],
    product_data['Waste disp. cost'],
    product_data['DSP storage cost'],
    product_data['Backlog penalty'],
    product_data['Changeover cost'],
    
    product_data['USP rate [days]'],
    product_data['USP lead [days]'],
    product_data['USP lifetime [days]'],
    product_data['USP cap. per 60 days'],
    
    product_data['DSP rate [days]'],
    product_data['DSP lead [days]'],
    product_data['DSP lifetime [days]'],
    product_data['DSP cap. per 60 days']
)

Run: 1, Best: 511.00


Run: 2, Best: 515.00


Run: 3, Best: 511.00


Run: 4, Best: 515.00


Run: 5, Best: 514.00


...

Run: 8, Best: 518.00


Run: 9, Best: 508.00


Run: 10, Best: 511.00


100%|█████████████████████████████████████████████████████████████████████████| 10000/10000 [00:08<00:00, 1151.86it/s]


Best solution found -> Profit: 518.00, Backlog penalty: 0.00


In [8]:
df = pd.DataFrame({key : [value] for key, value in example1_model.objectives.items()}, index=['value'])
df[['profit', 'sales', 'backlog_cost', 'production_cost', 'changeover_cost', 'dsp_storage_cost', 'dsp_waste_cost']]

,profit,sales,backlog_cost,production_cost,changeover_cost,dsp_storage_cost,dsp_waste_cost
value,518.0,680.0,0.0,136.0,10.0,16.0,0.0


In [9]:
gantt = make_gantt(example1_model.schedule, 'Base case')
py.iplot(gantt, filename='Lakhdar2005 Example1 Continuous-time Gantt', world_readable=True)

High five! You successfully sent some data to your account on plotly. View your plot in your browser at https://plot.ly/~karolis_j/0 or inside your plot.ly account where it is named 'Lakhdar2005 Example1 Continuous-time Gantt'


In [10]:
fig = make_demand_supply_chart(example1_model.sales_profile[0], demand_data.values[0], 'Product p1 Demand & Supply')
py.iplot(fig, filename='p1-demand-supply')

In [11]:
fig = make_demand_supply_chart(example1_model.sales_profile[1], demand_data.values[1], 'Product p2 Demand & Supply')
py.iplot(fig, filename='p2-demand-supply')

In [12]:
fig = make_demand_supply_chart(example1_model.sales_profile[2], demand_data.values[2], 'Product p3 Demand & Supply')
py.iplot(fig, filename='p3-demand-supply')

<a id='increased_p1_demand'></a>
## Increased demand for product p1
Demand for p1 at time period t6 is increased by 3 batches.

[back to index](#index)

In [13]:
increased_p1_demand = demand_data.copy().values
increased_p1_demand[0][-1] = 9
increased_p1_demand = pd.DataFrame(increased_p1_demand.tolist(), columns=demand_data.columns, index=demand_data.index)
qgrid.show_grid(increased_p1_demand)

Widget Javascript not detected.  It may not be installed or enabled properly.


In [14]:
example1_model = Example1Model().fit(
    num_usp_suites,
    num_dsp_suites,
    
    increased_p1_demand.values,
    days_per_period,
    
    product_data['USP storage cost'],
    product_data['Sales price'],
    product_data['Prod. cost'],
    product_data['Waste disp. cost'],
    product_data['DSP storage cost'],
    product_data['Backlog penalty'],
    product_data['Changeover cost'],
    
    product_data['USP rate [days]'],
    product_data['USP lead [days]'],
    product_data['USP lifetime [days]'],
    product_data['USP cap. per 60 days'],
    
    product_data['DSP rate [days]'],
    product_data['DSP lead [days]'],
    product_data['DSP lifetime [days]'],
    product_data['DSP cap. per 60 days']
)

Run: 1, Best: 562.00


Run: 2, Best: 562.00


Run: 3, Best: 535.00


Run: 4, Best: 561.00


Run: 5, Best: 562.00


...

Run: 8, Best: 563.00


Run: 9, Best: 562.00


Run: 10, Best: 562.00


100%|██████████████████████████████████████████████████████████████████████████| 10000/10000 [00:10<00:00, 917.73it/s]


Best solution found -> Profit: 563.00, Backlog penalty: 0.00


In [15]:
df = pd.DataFrame({key : [value] for key, value in example1_model.objectives.items()}, index=['value'])
df[['profit', 'sales', 'backlog_cost', 'production_cost', 'changeover_cost', 'dsp_storage_cost', 'dsp_waste_cost']]

,profit,sales,backlog_cost,production_cost,changeover_cost,dsp_storage_cost,dsp_waste_cost
value,563.0,740.0,0.0,148.0,12.0,17.0,0.0


In [16]:
gantt = make_gantt(example1_model.schedule, "Increased demand for p1")
py.iplot(gantt, filename='Lakhdar2005 Example1 Continuous-time Gantt', world_readable=True)

High five! You successfully sent some data to your account on plotly. View your plot in your browser at https://plot.ly/~karolis_j/0 or inside your plot.ly account where it is named 'Lakhdar2005 Example1 Continuous-time Gantt'


In [17]:
fig = make_demand_supply_chart(example1_model.sales_profile[0], increased_p1_demand.values[0], 'Product p1 Demand & Supply')
py.iplot(fig, filename='p1-demand-supply')

In [18]:
fig = make_demand_supply_chart(example1_model.sales_profile[1], increased_p1_demand.values[1], 'Product p2 Demand & Supply')
py.iplot(fig, filename='p2-demand-supply')

In [19]:
fig = make_demand_supply_chart(example1_model.sales_profile[2], increased_p1_demand.values[2], 'Product p3 Demand & Supply')
py.iplot(fig, filename='p3-demand-supply')

<a id='doubled_demand_profile'></a>
## Doubled demand profile
The demand profile from the original Example 1 problem is repeated for another year.

[back to index](#index)

In [20]:
import numpy as np

columns = ['t%d' % _ for _  in range(1, 13)]
doubled_demand_profile = pd.DataFrame(np.hstack([demand_data.values, demand_data.values]).tolist(), 
                                      columns=columns, 
                                      index=demand_data.index)

days_per_period_doubled = days_per_period + days_per_period

qgrid.show_grid(doubled_demand_profile)

Widget Javascript not detected.  It may not be installed or enabled properly.


In [21]:
example1_model = Example1Model().fit(
    num_usp_suites,
    num_dsp_suites,
    
    doubled_demand_profile.values,
    days_per_period_doubled,
    
    product_data['USP storage cost'],
    product_data['Sales price'],
    product_data['Prod. cost'],
    product_data['Waste disp. cost'],
    product_data['DSP storage cost'],
    product_data['Backlog penalty'],
    product_data['Changeover cost'],
    
    product_data['USP rate [days]'],
    product_data['USP lead [days]'],
    product_data['USP lifetime [days]'],
    product_data['USP cap. per 60 days'],
    
    product_data['DSP rate [days]'],
    product_data['DSP lead [days]'],
    product_data['DSP lifetime [days]'],
    product_data['DSP cap. per 60 days']
)

Run: 1, Best: 918.00


Run: 2, Best: 995.00


Run: 3, Best: 983.00


Run: 4, Best: 982.00


Run: 5, Best: 981.00


...

Run: 8, Best: 1005.00


Run: 9, Best: 960.00


Run: 10, Best: 964.00


100%|██████████████████████████████████████████████████████████████████████████| 10000/10000 [00:15<00:00, 628.13it/s]


Best solution found -> Profit: 1005.00, Backlog penalty: 0.00


In [22]:
gantt = make_gantt(example1_model.schedule, "Doubled demand profile")
py.iplot(gantt, filename='Lakhdar2005 Example1 Continuous-time Gantt', world_readable=True)

High five! You successfully sent some data to your account on plotly. View your plot in your browser at https://plot.ly/~karolis_j/0 or inside your plot.ly account where it is named 'Lakhdar2005 Example1 Continuous-time Gantt'


In [23]:
df = pd.DataFrame({key : [value] for key, value in example1_model.objectives.items()}, index=['value'])
df[['profit', 'sales', 'backlog_cost', 'production_cost', 'changeover_cost', 'dsp_storage_cost', 'dsp_waste_cost']]

,profit,sales,backlog_cost,production_cost,changeover_cost,dsp_storage_cost,dsp_waste_cost
value,1005.0,1360.0,0.0,274.0,30.0,51.0,0.0


In [24]:
fig = make_demand_supply_chart(example1_model.sales_profile[0], doubled_demand_profile.values[0], 'Product p1 Demand & Supply')
py.iplot(fig, filename='p1-demand-supply')

In [25]:
fig = make_demand_supply_chart(example1_model.sales_profile[1], doubled_demand_profile.values[1], 'Product p2 Demand & Supply')
py.iplot(fig, filename='p2-demand-supply')

In [26]:
fig = make_demand_supply_chart(example1_model.sales_profile[2], doubled_demand_profile.values[2], 'Product p3 Demand & Supply')
py.iplot(fig, filename='p3-demand-supply')

[back to index](#index)